# Emotion Classifier


## Setup and Training of Model

In [ ]:
# Imports

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import cv2

from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
pyplot.style.use(['dark_background'])

In [ ]:
df = pd.read_csv("fer2013/fer2013.csv")
df.shape

In [ ]:
# Emotion labels
df.emotion.unique()

In [ ]:
label_to_text = {0: "anger",
                    1: "disgust",
                    2: "fear",
                    3: "happiness",
                    4: "sadness",
                    5: "surprise",
                    6: "neutral"}

In [ ]:
# Convert string inputs of pixels to 48x48 grid for images
first_img = df.pixels.loc[0]
first_grid = np.array(first_img.split(" ")).reshape(48, 48)
first_grid

In [ ]:
# Display image using pyplot
pyplot.imshow(first_grid.astype('float'))

In [ ]:
# Display some sample data
fig = pyplot.figure(1, (14, 14))
k = 0
for label in sorted(df.emotion.unique()):
    for j in range(3):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(" ")).reshape(48, 48).astype("float32")
        k += 1
        ax = pyplot.subplot(7, 7, k)
        ax.imshow(px)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label_to_text[label])
        pyplot.tight_layout()

In [ ]:
# Apply conversion function to all rows of data
img_array = df.pixels.apply(lambda x : np.array(x.split(" ")).reshape(48, 48, 1).astype("float32"))

In [ ]:
# Arrange array
img_array = np.stack(img_array, axis = 0)
img_array.shape

In [ ]:
# Obtain array of emotion labels
labels = df.emotion.values
labels

In [ ]:
# Split dataset into training and testing data
x_train, x_test, y_train, y_test = train_test_split(img_array, labels, test_size = 0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# Normalise data (images)
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
# Create Sequential Model
base_model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(filters=32, 
                                                               kernel_size=(3, 3), 
                                                               activation='relu', 
                                                               input_shape=(48,48,1)),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        tf.keras.layers.BatchNormalization(),

                                        tf.keras.layers.Conv2D(filters=64, 
                                                               kernel_size=(3, 3), 
                                                               activation='relu', 
                                                               input_shape=(48,48,1)),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        tf.keras.layers.BatchNormalization(),

                                        tf.keras.layers.Conv2D(filters=128, 
                                                               kernel_size=(3, 3), 
                                                               activation='relu', 
                                                               input_shape=(48,48,1)),
                                        tf.keras.layers.MaxPool2D(2,2),
                                        tf.keras.layers.BatchNormalization(),

                                        tf.keras.layers.Flatten(),
                                        tf.keras.layers.Dense(units=128, 
                                                              activation='relu'),
                                        tf.keras.layers.Dropout(0.5),
                                        tf.keras.layers.Dense(units=7, 
                                                              activation='softmax') # 7 emotion classes

])
base_model.summary()

In [ ]:
base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss='sparse_categorical_crossentropy', # Sparse as labels are integers (one-hot encoding)
                   metrics=['accuracy'])

In [ ]:
# Make directory for storing model checkpoints
try:
    os.mkdir('checkpoints')
except:
    pass

In [ ]:
file_name = 'best_model.keras'
checkpoint_path = os.path.join('checkpoints', file_name) 

call_back = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               monitor='val_accuracy',
                                               verbose=1,
                                               save_freq='epoch',
                                               save_best_only=True,
                                               save_weights_only=False,
                                               mode='max')

In [ ]:
# Start model training

base_model.fit(x=x_train, 
               y=y_train,
               epochs=20,
               validation_split=0.2,
               callbacks=call_back)

In [ ]:
# Check model performance against test data

best_model = tf.keras.models.load_model(checkpoint_path)

for k in range(10):
    print(f'actual label is {label_to_text[y_test[k]]}')
    predicted_class = best_model.predict(tf.expand_dims(x_test[k], axis=0)).argmax()
    print(f'predicted label is {label_to_text[predicted_class]}')
    pyplot.imshow(x_test[k].reshape(48, 48))
    pyplot.show()
    time.sleep(1)
    clear_output(wait=True)

## Real-time prediction via webcam

In [ ]:
# Set up face classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Set up labels
label_to_text = {0: "anger",
                    1: "disgust",
                    2: "fear",
                    3: "happiness",
                    4: "sadness",
                    5: "surprise",
                    6: "neutral"}

# Obtain best model
file_name = 'best_model.keras'
checkpoint_path = os.path.join('checkpoints', file_name) 
best_model = tf.keras.models.load_model(checkpoint_path)

# Access webcam
video_capture = cv2.VideoCapture(0)

In [ ]:
# Function to classify emotions in video stream
def detect_classify_emotion(vid):
    # convert to grayscale for processing
    img_gray = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)

    # detect faces available in video stream
    faces = face_classifier.detectMultiScale(img_gray, 1.1, 5, minSize = (40, 40))

    # process each available face in video stream
    for (x, y, w, h) in faces:
        # draw bounding box around face
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)

        # pre-process face for model prediction
        face = img_gray[y:y + h, x:x + w]
        cropped_face = np.expand_dims(np.expand_dims(cv2.resize(face, (48, 48)), -1), 0)

        # preduct emotion using best model
        predicted_emotion = best_model.predict(cropped_face)
        max_index = int(np.argmax(predicted_emotion))
        cv2.putText(vid, label_to_text[max_index], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    return faces


In [ ]:

# Use a loop for real-time face detection
while True:

    result, video_frame = video_capture.read() # read frames from video
    if result is False:
        break # terminate the loop if the frame is not read successfully

    faces = detect_classify_emotion(video_frame)

    cv2.imshow("Dylan's Emotion Classifier Project", video_frame) # display processed frame in window

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()